# Yolo Tutorial

In [ ]:
CNF={'id':f'yolo0620x',
     'train':False,
     'checkPredcitions':True,
     'infer':False,
     'createPseudo':True,
     'train_jigsaw':False,
     'balance_sampling':False,
     'max_epoch':100,#100
     'fold':2,
     'exdata0a':False,
     'efd_level':0,
     'save_best_ckpt_num':1,
     'frac':1,
     'target_fold':1, # いつもは0
     'cutmix':False,
     'batch_size':4,#4*3 728->4
     'mixup_last_n':1, # 0 -> no mixup 素のbatchsizeは12の時は3が丁度良かった
     'aws':False,
     
     'lastCKPT':f"../input/large0618fold1/*last*", # CNFにlastCKPTキーがあれば続きからトレーニング
     'bestCKPT':f"../input/large0618fold1/*best*", # CNFにbestCKPTキーがあれば読み込んで性能評価、Inferとか
#      'lastCKPT':f"../input/trainovermergedimage5fold/*last*", # CNFにlastCKPTキーがあれば続きからトレーニング
#      'bestCKPT':f"../input/trainovermergedimage5fold/*best*", # CNFにbestCKPTキーがあれば読み込んで性能評価、Inferとか
     'SEED':42
     }


YoloのInstall

In [ ]:
%%capture
!git clone https://github.com/ultralytics/yolov5  # clone repo
!pip install -r yolov5/requirements.txt  # install dependencies
#%cd yolov5

In [ ]:
import numpy as np
import pandas as pd
import os
from ast import literal_eval

train_df = pd.read_csv("../input/global-wheat-detection/train.csv")

In [ ]:
def convert(size, box):
    dw = 1. / size[0];dh = 1. / size[1];x = (box[0] + box[1]) / 2.0;y = (box[2] + box[3]) / 2.0
    w = box[1] - box[0];h = box[3] - box[2]
    x = x * dw;w = w * dw;y = y * dh;h = h * dh
    return [x, y, w, h]

def convert_to_yolo_label(coco_format_box, w = 1024, h = 1024):
    bbox = literal_eval(coco_format_box)
    xmin = bbox[0];xmax = bbox[0] + bbox[2];ymin = bbox[1];ymax = bbox[1] + bbox[3]
    b = (float(xmin), float(xmax), float(ymin), float(ymax))
    yolo_box = convert((w, h), b)
    if np.max(yolo_box) > 1 or np.min(yolo_box) < 0: # Take this opportunity to check that conversion works
        print("BOX HAS AN ISSUE")
    return yolo_box

train_df['yolo_box'] = train_df.bbox.apply(convert_to_yolo_label)

In [ ]:
from sklearn.model_selection import StratifiedKFold
def append_folds(marking):

    skf = StratifiedKFold(n_splits=CNF['fold'], shuffle=True, random_state=42)
        
    df_folds = marking[['image_id']].copy()
    df_folds.loc[:, 'bbox_count'] = 1
    df_folds = df_folds.groupby('image_id').count()
    df_folds.loc[:, 'source'] = marking[['image_id', 'source']].groupby('image_id').min()['source']
    df_folds.loc[:, 'stratify_group'] = np.char.add(
        df_folds['source'].values.astype(str),
        df_folds['bbox_count'].apply(lambda x: f'_{x // 15}').values.astype(str)
    )
    df_folds.loc[:, 'fold'] = 0

    for fold_number, (train_index, val_index) in enumerate(skf.split(X=df_folds.index, y=df_folds['stratify_group'])):
        df_folds.loc[df_folds.iloc[val_index].index, 'fold'] = fold_number

    df_folds=df_folds.reset_index()

    marking=marking.merge(df_folds[['image_id','fold','stratify_group']],how='left',on='image_id')
        
        
    return marking
    

In [ ]:
# bboxなし画像IDリストnegative_image_ids
from glob import glob
all_imgs = glob("../input/global-wheat-detection/train/*.jpg")
all_imgs = [i.split("/")[-1].replace(".jpg", "") for i in all_imgs]
positive_imgs = train_df.image_id.unique()
negative_image_ids = list(set(all_imgs) - set(positive_imgs))

In [ ]:
train_df=append_folds(train_df) if 'fold' not in train_df.columns else train_df

# subsample
if CNF['frac']!=1:
    image_list = pd.DataFrame(train_df.image_id.unique()).sample(frac=CNF['frac'],random_state=42).values.squeeze()
    train_df   = train_df[train_df.image_id.isin(image_list)]

# ホームディレクトリにて環境設定

In [ ]:
import shutil

if os.path.basename(os.getcwd())=='yolov5':
    os.chdir('..')  
    
# working---yolov5---data---GWHDfold0.yaml
#         |               |-GWHDfold1.yaml
#         |
#         |-gwhd---images---trn_fold0--many jpgs
#                |        |-val_fold0--many jpgs
#                |        |-trn_fold1--many jpgs...
#                |        |-val_fold1--many jpgs...
#                |
#                |-labels---trn_fold0--many txt (bbox)
#                         |-val_fold0--many txt

#
for fold in range(CNF['fold']):
    
    !echo -e "train: ../gwhd/images/trn_fold{fold}\nval: ../gwhd/images/val_fold{fold}\nnc: 1\nnames: ['wheat']" > ./yolov5/data/GWHDfold{fold}.yaml
    !mkdir -p ./gwhd/images/trn_fold{fold} ./gwhd/images/val_fold{fold} ./gwhd/labels/trn_fold{fold} ./gwhd/labels/val_fold{fold}
    
    trn_images=train_df[train_df.fold!=fold].image_id.unique()
    val_images=train_df[train_df.fold==fold].image_id.unique()
    
    [ shutil.copyfile(f"../input/global-wheat-detection/train/{image_id}.jpg",f"./gwhd/images/trn_fold{fold}/{image_id}.jpg") for image_id in trn_images ]
    [ shutil.copyfile(f"../input/global-wheat-detection/train/{image_id}.jpg",f"./gwhd/images/val_fold{fold}/{image_id}.jpg") for image_id in val_images ]
    [ shutil.copyfile(f"../input/global-wheat-detection/train/{image_id}.jpg",f"./gwhd/images/trn_fold{fold}/{image_id}.jpg") for image_id in negative_image_ids ]
    
    for trn_or_val,image_ids in zip(['trn','val'],[trn_images,val_images]):
        
        for img_id in image_ids:
            filt_df = train_df.query("image_id == @img_id") 
            all_boxes = filt_df.yolo_box.values
            if os.path.exists(f"./gwhd/labels/{trn_or_val}_fold{fold}/{img_id}.txt")==False:
                with open(f"./gwhd/labels/{trn_or_val}_fold{fold}/{img_id}.txt", 'a') as file: 
                    for i in all_boxes:
                        new_line = ("0 %s %s %s %s \n" % tuple(i));file.write(new_line)  

    for img_id in negative_image_ids:

        with open(f"./gwhd/labels/trn_fold{fold}/{img_id}.txt", 'w') as fp: 
            pass


In [ ]:
if os.path.basename(os.getcwd())!='yolov5':
    os.chdir('yolov5')  
    
for fold in range(CNF['fold'])[:1]:
    
    # クラス数を1に設定
    !sed "s/nc: 80/nc: 1/" ../yolov5/models/yolov5s.yaml > ../yolov5/models/yolov5s_{CNF['id']}_fold{fold}.yaml
    
    !python train.py --img 1024 --batch 16 --epochs {CNF['max_epoch']} --data ./data/GWHDfold{fold}.yaml --cfg ./models/yolov5s_{CNF['id']}_fold{fold}.yaml --weights yolov5s.pt --name {CNF['id']}_fold{fold} --nosave --cache

In [ ]:
cp weights/last_{CNF['id']}_fold{fold}.pt ..

In [ ]:
!cd ..
!rm -rf ./gwhd
!rm -rf ./yolov5

In [ ]:
# !python detect.py --weights ./weights/last_xlarge0619_fold0.pt --img 1024 --conf 0.5 --source ../gwhd/images/val_fold0


In [ ]:
from IPython.display import Image
fnames =glob("inference/output/*")
Image(fnames[50],width=512)

## Albumentations

In [ ]:
def get_train_transforms():
    return A.Compose(
        [   
            A.RandomSizedCrop(min_max_height=(800, 800), height=1024, width=1024, p=0.5),
            A.OneOf([
                A.HueSaturationValue(hue_shift_limit=0.2, sat_shift_limit= 0.2, val_shift_limit=0.2, p=0.9),
                A.RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.2, p=0.9),],p=0.9),
            A.ToGray(p=0.01),
            A.HorizontalFlip(p=0.5),
            A.VerticalFlip(p=0.5),
            A.Resize(height=CNF['nnimSize'], width=CNF['nnimSize'], p=1),
            A.Cutout(num_holes=8, max_h_size=64, max_w_size=64, fill_value=0, p=0.5),#これはCNF['nnimSize']かえても変えてない
            ToTensorV2(p=1.0),
        ], 
        p=1.0, 
        bbox_params=A.BboxParams(format='pascal_voc',min_area=0,min_visibility=0,label_fields=['labels'])
    )

def get_valid_transforms():
    return A.Compose([A.Resize(height=CNF['nnimSize'], width=CNF['nnimSize'], p=1.0),ToTensorV2(p=1.0),], p=1.0, 
        bbox_params=A.BboxParams(format='pascal_voc',min_area=0, min_visibility=0,label_fields=['labels']))

def get_test_transforms():
    return A.Compose([A.Resize(height=CNF['nnimSize'], width=CNF['nnimSize'], p=1.0),ToTensorV2(p=1.0)])


## Dataset

In [ ]:
class DatasetRetriever(Dataset):

    def __init__(self, jigsaw, marking, transforms=None, test=False):
        super().__init__()
        
        # check df index is 0,1,2,...
        assert np.all( marking.index.tolist()==np.arange(len(marking)) )
        self.jigsaw=jigsaw ; self.marking=marking ; self.transforms=transforms ; self.test=test
        
        self.image_ids = marking.merge_id.drop_duplicates().values if jigsaw else marking.image_id.drop_duplicates().values
        
        if jigsaw:
            self.markingPerMergeId = dict(tuple(marking.groupby('merge_id')))
        self.markingPerImageId = dict(tuple(marking.groupby('image_id')))
        
        self.infer = not ("x" in marking.columns) # test prediction mode
        
    def __getitem__(self, index: int):
        
        image_id = self.image_ids[index]
        
        if self.infer:
            image=readImage(image_id)
            return self.transforms(**{'image': image})['image'],image_id # test prediction mode
            
        if CNF['cutmix']:
            if self.test or random.random() > 0.5:
                image, boxes = self.load_image_and_boxes(index)
            else:
                image, boxes = self.load_cutmix_image_and_boxes(index)
        else:
            image, boxes = self.load_image_and_boxes(index)
            
        # there is only one class
        labels = torch.ones((boxes.shape[0],), dtype=torch.int64)
        
        target = {}
        target['boxes'] = boxes
        target['labels'] = labels
        target['image_id'] = torch.tensor([index])

        if self.transforms:
            for i in range(10): # トランスフォームしすぎてBBOX無くなる場合を避ける
                sample = self.transforms(**{'image': image,'bboxes': target['boxes'],'labels': labels})
                if len(sample['bboxes']) > 0:
                    image = sample['image']
                    target['boxes'] = torch.stack(tuple(map(torch.tensor, zip(*sample['bboxes'])))).permute(1, 0)
                    target['boxes'][:,[0,1,2,3]] = target['boxes'][:,[1,0,3,2]]  #yxyx: be warning
                    break

        return image, target, image_id

    def __len__(self) -> int:
        return len(self.image_ids) 
        
    def load_image_and_boxes(self, index):
        image_id = self.image_ids[index]
        
        if self.jigsaw:
            
            # 画像統合
            _mgdf=merge_id2image_id_df[image_id] # ここでいう引数のimage_idはmerge_idのことね
            _ims=[ readImage(image_id) for image_id in _mgdf.image_id]
            if len(_ims)==6: # _mgdf.NUM_PATCH_SRCと同じ
                image=np.vstack([np.hstack([_ims[0],_ims[1],_ims[2]]),np.hstack([_ims[3],_ims[4],_ims[5]])])
                #start_tate, start_yoko = np.random.randint(0, 1024), np.random.randint(0, 2048)
            elif len(_ims)==4:
                image=np.vstack([np.hstack([_ims[0],_ims[1]]),np.hstack([_ims[2],_ims[3]])])
                #start_tate, start_yoko = np.random.randint(0, 1024), np.random.randint(0, 1024)
            else:
                image=np.hstack([_ims[0],_ims[1]])
                #start_tate, start_yoko = 0, np.random.randint(0, 1024)
                
            records = self.markingPerMergeId[image_id] # ここのインデックスはmerge_idの意味
            boxes = records[['mb0','mb1','mb2','mb3']].values
               
            #NO_USE_ALBUM
            if False:
                # 1024x1024 crop
                image=image[start_tate:(start_tate+1024),start_yoko:(start_yoko+1024),:]
                # box crop
                boxes[:,0]-=start_yoko;boxes[:,1]-=start_tate;boxes[:,2]-=start_yoko;boxes[:,3]-=start_tate
                boxes=boxes.clip(0,1023)
                boxes=boxes[ (boxes[:,2]-boxes[:,0])*(boxes[:,3]-boxes[:,1]) != 0 ]              
            
        else:
            
            image=readImage(image_id)
            records = self.markingPerImageId[image_id]
            boxes = records[['x', 'y', 'w', 'h']].values
            boxes[:, 2] = boxes[:, 0] + boxes[:, 2]
            boxes[:, 3] = boxes[:, 1] + boxes[:, 3]
            
        return image, boxes

    def load_cutmix_image_and_boxes(self, index, imsize=1024):
        
        # 画面四分割して左上はindexの画像、他３つは適当にとってきた画像
        """ 
        This implementation of cutmix author:  https://www.kaggle.com/nvnnghia 
        Refactoring and adaptation: https://www.kaggle.com/shonenkov
        """
        w, h = imsize, imsize
        s = imsize // 2
    
        xc, yc = [int(random.uniform(imsize * 0.25, imsize * 0.75)) for _ in range(2)]  # center x, y
        indexes = [index] + [random.randint(0, self.image_ids.shape[0] - 1) for _ in range(3)]

        result_image = np.full((imsize, imsize, 3), 1, dtype=np.float32) # まず白で埋めた画像をつくる
        result_boxes = []

        for i, index in enumerate(indexes):
            image, boxes = self.load_image_and_boxes(index)
            if i == 0:
                x1a, y1a, x2a, y2a = max(xc - w, 0), max(yc - h, 0), xc, yc  # xmin, ymin, xmax, ymax (large image)
                x1b, y1b, x2b, y2b = w - (x2a - x1a), h - (y2a - y1a), w, h  # xmin, ymin, xmax, ymax (small image)
            elif i == 1:  # top right
                x1a, y1a, x2a, y2a = xc, max(yc - h, 0), min(xc + w, s * 2), yc
                x1b, y1b, x2b, y2b = 0, h - (y2a - y1a), min(w, x2a - x1a), h
            elif i == 2:  # bottom left
                x1a, y1a, x2a, y2a = max(xc - w, 0), yc, xc, min(s * 2, yc + h)
                x1b, y1b, x2b, y2b = w - (x2a - x1a), 0, max(xc, w), min(y2a - y1a, h)
            elif i == 3:  # bottom right
                x1a, y1a, x2a, y2a = xc, yc, min(xc + w, s * 2), min(s * 2, yc + h)
                x1b, y1b, x2b, y2b = 0, 0, min(w, x2a - x1a), min(y2a - y1a, h)
            result_image[y1a:y2a, x1a:x2a] = image[y1b:y2b, x1b:x2b]
            padw = x1a - x1b
            padh = y1a - y1b

            boxes[:, 0] += padw
            boxes[:, 1] += padh
            boxes[:, 2] += padw
            boxes[:, 3] += padh

            result_boxes.append(boxes)

        result_boxes = np.concatenate(result_boxes, 0)
        np.clip(result_boxes[:, 0:], 0, 2 * s, out=result_boxes[:, 0:])
        result_boxes = result_boxes.astype(np.int32)
        result_boxes = result_boxes[np.where((result_boxes[:,2]-result_boxes[:,0])*(result_boxes[:,3]-result_boxes[:,1]) > 0)]
        return result_image, result_boxes

In [ ]:
# marking = append_folds(marking,is_jigsaw_group_split=CNF['train_jigsaw'])
# marking = marking.dropna()
# fold_number=0
# train_dataset = DatasetRetriever(
#         jigsaw=CNF['train_jigsaw'],
#         marking=marking[marking['fold'] != fold_number].reset_index(drop=True),
#         transforms=get_train_transforms(),
#         test=False,
#     )

# for  cnt,(image, target, image_id) in enumerate(train_dataset):

#     plt.figure(figsize=(10,10))
#     image=image.permute(1, 2, 0).numpy()
#     for box in target['boxes'].numpy():

#         box=box.astype('int')
#         cv2.rectangle(image, (box[1], box[0]), (box[3], box[2]), (1, 1, 1), 2) 

#     plt.imshow(image)
    
#     if cnt>100:
#         break

In [ ]:
def createDataSet(marking):

    fold_number = CNF['target_fold']

    train_dataset = DatasetRetriever(
        jigsaw=CNF['train_jigsaw'],
        marking=marking[marking['fold'] != fold_number].reset_index(drop=True),
        transforms=get_train_transforms(),
        test=False,
    )

    validation_dataset = DatasetRetriever(
        jigsaw=False,
        marking=marking[marking['fold'] == fold_number].reset_index(drop=True),
        transforms=get_valid_transforms(),
        test=True,
    )
    
    return train_dataset, validation_dataset,None
    

## Fitter

In [ ]:
class Fitter:
    
    def __init__(self, model, device, config):
        
        self.hist={'trn_loss':[],'val_loss':[],'tst_loss':[]}
        
        self.config = config
        self.epoch = 0

        self.base_dir = f'./{config.folder}'
        if not os.path.exists(self.base_dir):
            os.makedirs(self.base_dir)
        
        self.log_path = f'{self.base_dir}/log.txt'
        self.best_summary_loss = 10**5

        self.model = model
        self.device = device

        param_optimizer = list(self.model.named_parameters())
        no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
        optimizer_grouped_parameters = [
            {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': 0.001},
            {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
        ] 

        self.optimizer = torch.optim.AdamW(self.model.parameters(), lr=config.lr)
        self.scheduler = config.SchedulerClass(self.optimizer, **config.scheduler_params)
        self.log(f'Fitter prepared. Device is {self.device}')

    def fit(self, train_loader, validation_loader):
        
        for e in range(self.config.n_epochs):
            if self.config.verbose:
                lr = self.optimizer.param_groups[0]['lr']
                timestamp = datetime.utcnow().isoformat()
                self.log(f'\n{timestamp}\nLR: {lr}')

            t = time.time()
            summary_loss = self.train_one_epoch(train_loader)
            self.hist['trn_loss'].append(summary_loss.avg)
            self.log(f'[RESULT]: Train. Epoch: {self.epoch}, summary_loss: {summary_loss.avg:.5f}, time: {(time.time() - t):.5f}')
            #if e==(self.config.n_epochs-1):
            self.save(f'{self.base_dir}/last-checkpoint.bin') # 毎回保存は処理が重いかな

            t = time.time()
            summary_loss = self.validation(validation_loader)
            self.hist['val_loss'].append(summary_loss.avg)
            self.log(f'[RESULT]: Val. Epoch: {self.epoch}, summary_loss: {summary_loss.avg:.5f}, time: {(time.time() - t):.5f}')
            if summary_loss.avg < self.best_summary_loss:
                self.best_summary_loss = summary_loss.avg
                self.model.eval()
                self.save(f'{self.base_dir}/best-checkpoint-{str(self.epoch).zfill(3)}epoch.bin')
                for path in sorted(glob(f'{self.base_dir}/best-checkpoint-*epoch.bin'))[:-CNF['save_best_ckpt_num']]:os.remove(path)

            if self.config.validation_scheduler:
                self.scheduler.step(metrics=summary_loss.avg)

            self.epoch += 1
                
        return self.hist

    def validation(self, val_loader,is_test=False,return_loss=False):
        self.model.eval()
        summary_loss = AverageMeter()
        t = time.time()
        for step, (images, targets, image_ids) in enumerate(val_loader):
            if self.config.verbose:
                if step % self.config.verbose_step == 0:
                    ValOrTst = '** Test **' if is_test else 'Val'
                    print(f'{ValOrTst} Step {step}/{len(val_loader)}, ' + f'summary_loss: {summary_loss.avg:.5f}, ' + f'time: {(time.time() - t):.5f}', end='\r')
                    
            with torch.no_grad():
                
                images = torch.stack(images)
                batch_size = images.shape[0]
                images = images.to(self.device).float()
                boxes  = [target['boxes'].to(self.device).float()  for target in targets]
                labels = [target['labels'].to(self.device).float() for target in targets]

                loss, _, _ = self.model(images, boxes, labels)
                summary_loss.update(loss.detach().item(), batch_size)

        if return_loss:
            return summary_loss,loss
        else:
            return summary_loss

    def train_one_epoch(self, train_loader):
        
        self.scheduler.step()
        
        self.model.train()
        summary_loss = AverageMeter()
        t = time.time()
        for step, (images, targets, image_ids) in enumerate(train_loader):
            if self.config.verbose:
                if step % self.config.verbose_step == 0:
                    print(f'Train Step {step}/{len(train_loader)}, ' + f'summary_loss: {summary_loss.avg:.5f}, ' + f'time: {(time.time() - t):.5f}', end='\r')
            
            images = torch.stack(images)
            images = images.to(self.device).float()
            batch_size = images.shape[0]
            boxes  = [target['boxes'].to(self.device).float()  for target in targets]
            labels = [target['labels'].to(self.device).float() for target in targets]

            self.optimizer.zero_grad()
            
            loss, _, _ = self.model(images, boxes, labels, num_mixup=CNF['mixup_last_n'])
            
            loss.backward()

            summary_loss.update(loss.detach().item(), batch_size)

            self.optimizer.step()

            if self.config.step_scheduler:
                self.scheduler.step()

        return summary_loss
    
    def save(self, path):
        self.model.eval()
        torch.save({
            'model_state_dict': self.model.model.state_dict(),'optimizer_state_dict': self.optimizer.state_dict(),
            'scheduler_state_dict': self.scheduler.state_dict(),'best_summary_loss': self.best_summary_loss,
            'epoch': self.epoch,}, path)

    def load(self, path):
        checkpoint = torch.load(path, map_location=device) # GPUの時にチェックしてない
        self.model.model.load_state_dict(checkpoint['model_state_dict'])
        self.optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        self.scheduler.load_state_dict(checkpoint['scheduler_state_dict'])
        self.best_summary_loss = checkpoint['best_summary_loss']
        self.epoch = checkpoint['epoch'] + 1
        
    def predict(self, loader,score_threshold,isTrn=True):
        self.model.eval();predictions=[]
        for images, targets, image_ids in loader:
                
            with torch.no_grad():
                
                images = torch.stack(images);batch_size = images.shape[0];images = images.to(self.device).float()
                det = self.model(images, torch.tensor([1]*images.shape[0]).float().to(device))

                for i in range(images.shape[0]):
                    boxes = det[i].detach().cpu().numpy()[:,:4];scores = det[i].detach().cpu().numpy()[:,4]
                    indexes = np.where(scores >= score_threshold)[0]
                    boxes = boxes[indexes];boxes[:, 2] = boxes[:, 2] + boxes[:, 0];boxes[:, 3] = boxes[:, 3] + boxes[:, 1]
                    predictions.append({ 'boxes': boxes[indexes],'scores': scores[indexes],})

        return predictions
    
    def log(self, message):
        if self.config.verbose:
            print(message)
        with open(self.log_path, 'a+') as logger:
            logger.write(f'{message}\n')

In [ ]:
class TrainGlobalConfig:
 
    num_workers = 8 if CNF['aws'] else 2 
    batch_size = CNF['batch_size']
    n_epochs = CNF['max_epoch']# n_epochs = 40
    lr = 1
    folder = f'effdet-{CNF["id"]}'
    verbose = True
    verbose_step = 1
    step_scheduler = False  # do scheduler.step after optimizer.step
    validation_scheduler = False  # do scheduler.step after validation stage loss
    
    import functools
    
    def mylr(epch):
        
        base=0.0001
        _lr = [10]*4+[5]*4+[1]*4
        CYCLE_LEN=len(_lr)
        return _lr[epch%CYCLE_LEN]*base
    
    SchedulerClass = torch.optim.lr_scheduler.LambdaLR
    
    scheduler_params = dict(lr_lambda = mylr,)
    

In [ ]:
def collate_fn(batch):
    return tuple(zip(*batch))

def run_training(net,train_dataset,validation_dataset,test_dataset,trn_sample_weight):
    
    net.to(device)
    
    trn_sampler=WeightedRandomSampler(trn_sample_weight,len(trn_sample_weight),replacement=True) if CNF['balance_sampling'] else RandomSampler(train_dataset)

    train_loader = torch.utils.data.DataLoader(
        train_dataset,
        batch_size=TrainGlobalConfig.batch_size,
        sampler=trn_sampler,
        pin_memory=False,
        drop_last=True,
        num_workers=TrainGlobalConfig.num_workers,
        collate_fn=collate_fn,
    )
    val_loader = torch.utils.data.DataLoader(
        validation_dataset, 
        batch_size=TrainGlobalConfig.batch_size,
        num_workers=TrainGlobalConfig.num_workers,
        shuffle=False,
        sampler=SequentialSampler(validation_dataset),
        pin_memory=False,
        collate_fn=collate_fn,
    )
    
    fitter = Fitter(model=net, device=device, config=TrainGlobalConfig)
    
    if 'lastCKPT' in CNF.keys():
        fitter.load(glob(CNF['lastCKPT'])[0]) # 続きからトレーニング
    
    train_history=fitter.fit(train_loader, val_loader)
    
    return train_history

In [ ]:
from effdet import get_efficientdet_config,   DetBenchEval, EfficientDet #,DetBenchTrain,
from effdet.efficientdet import HeadNet
from torch import nn
class GWDModel(nn.Module):

    def __init__(self,isTrn=False):
        super(GWDModel, self).__init__()
        
        self.config = get_efficientdet_config(f'tf_efficientdet_d{CNF["efd_level"]}')
        self.net = EfficientDet(self.config, pretrained_backbone=False)
        checkpoint = torch.load(glob(f'../input/efficientdet/efficientdet_d{CNF["efd_level"]}*.pth')[0]) 
        self.net.load_state_dict(checkpoint)
        self.config.num_classes = 1
        self.config.image_size = CNF['nnimSize']
        self.net.class_net = HeadNet(self.config, num_outputs=self.config.num_classes, norm_kwargs=dict(eps=.001, momentum=.01))
        
    def forward(self, x):
        
        return self.net(x)
        

In [ ]:
def get_net(isTrn=False):
    
    net = GWDModel(isTrn=isTrn)
    
    return DetBenchTrain(net, net.config) if isTrn else DetBenchEval(net, net.config)

In [ ]:
from effdet.anchors import Anchors, AnchorLabeler, generate_detections, MAX_DETECTION_POINTS
from effdet.loss import DetectionLoss

# from timm import create_model

# class EfficientDet(nn.Module):

#     def __init__(self, config, norm_kwargs=None, pretrained_backbone=True):
#         super(EfficientDet, self).__init__()
#         norm_kwargs = norm_kwargs or dict(eps=.001, momentum=.01)
#         self.backbone = create_model(
#             config.backbone_name, features_only=True, out_indices=(2, 3, 4),
#             pretrained=pretrained_backbone, **config.backbone_args)
#         feature_info = [dict(num_chs=f['num_chs'], reduction=f['reduction'])
#                         for i, f in enumerate(self.backbone.feature_info())]
#         self.fpn = BiFpn(config, feature_info, norm_kwargs=norm_kwargs)
#         self.class_net = HeadNet(config, num_outputs=config.num_classes, norm_kwargs=norm_kwargs)
#         self.box_net = HeadNet(config, num_outputs=4, norm_kwargs=norm_kwargs)

#         for n, m in self.named_modules():
#             if 'backbone' not in n:
#                 _init_weight(m, n)

#     def forward(self, x):
#         x = self.backbone(x)
#         x = self.fpn(x)
#         x_class = self.class_net(x)
#         x_box = self.box_net(x)
#         return x_class, x_box    


class DetBenchTrain(nn.Module):
    def __init__(self, model, config):
        super(DetBenchTrain, self).__init__()
        self.config = config;self.model = model
        anchors = Anchors(config.min_level, config.max_level,config.num_scales, config.aspect_ratios,config.anchor_scale, config.image_size)
        self.anchor_labeler = AnchorLabeler(anchors, config.num_classes, match_threshold=0.5)
        self.loss_fn = DetectionLoss(self.config)

    def forward(self, x, gt_boxes, gt_labels, num_mixup=0):
                
        # 追加するパターン
        for i in range(  num_mixup ):

            x = torch.cat([x, (0.5*x[2*i,:,:,:]+0.5*x[2*i+1,:,:,:]).unsqueeze(0)])
            gt_boxes.append( torch.cat([gt_boxes[2*i] ,gt_boxes[2*i+1]] ) )
            gt_labels.append(torch.cat([gt_labels[2*i],gt_labels[2*i+1]]) )

        class_out, box_out = self.model(x)

        cls_targets = []
        box_targets = []
        num_positives = []
        # FIXME this may be a bottleneck, would be faster if batched, or should be done in loader/dataset?
        for i in range(x.shape[0]):
            gt_class_out, gt_box_out, num_positive = self.anchor_labeler.label_anchors(gt_boxes[i], gt_labels[i])
            cls_targets.append(gt_class_out)
            box_targets.append(gt_box_out)
            num_positives.append(num_positive)

        return self.loss_fn(class_out, box_out, cls_targets, box_targets, num_positives)

In [ ]:
def run_experiment(marking):
    
    marking = append_folds(marking,is_jigsaw_group_split=CNF['train_jigsaw'])
    marking = marking.dropna() # jigsawしてない画像は「今回は外す、考慮しない、全部jigsawできた」とする
    
    # subsample
    if CNF['frac']!=1:
        image_list = pd.DataFrame(marking.image_id.unique()).sample(frac=CNF['frac'],random_state=42).values.squeeze()
        marking = marking[marking.image_id.isin(image_list)]

    
    train_dataset, validation_dataset, test_dataset = createDataSet(marking)
    
    net = get_net(isTrn=True)
    
    if CNF['balance_sampling']:
        
        _tmp=marking[['merge_id','fold','trn_weight']].drop_duplicates()
        trn_sample_weight=_tmp.loc[_tmp['fold']!=0,'trn_weight']
    
    else:
        trn_sample_weight=None
    
    train_history=run_training(net, train_dataset, validation_dataset,test_dataset,trn_sample_weight)
    
    _save(train_history,f'history_{CNF["id"]}.pkl')
    
    fig, ax = plt.subplots(1, 1, figsize=(16, 8))

    ax.plot(np.array(train_history['trn_loss']),label='trn',color='blue')
    ax.plot(np.array(train_history['val_loss']),label='val',color='red')
    ax.legend()
    title_txt=str(CNF).translate(str.maketrans({"'":"","}":"","{":""}))
    ax.set_title(title_txt);ax.set_ylim(0, 1)
    return train_history

    

# Experiment

In [ ]:
# if CNF['train']:

#     result_df=pd.DataFrame(np.zeros((1,1)),columns=['experiment'],index=['loss'])

#     history=run_experiment(marking.copy())
    
#     result_df.loc['loss','experiment'] = history['val_loss'][-1]#history['tst_loss'][-1]
#     result_df.to_csv('result_df.csv',index=False)
#     for ep,v in enumerate(history['val_loss']):
#         print(f"EP{ep:03d}:{v:.5f}")
#     result_df

# Check Predictions

In [ ]:
# if CNF['checkPredcitions']:

#     marking = append_folds(marking,is_jigsaw_group_split=CNF['train_jigsaw'])
#     marking = marking.dropna() # jigsawしてない画像は「今回は外す、考慮しない、全部jigsawできた」とする
    
#     # 外部データ入れるとバグあるから応急処置
#     marking=marking[marking.source!='ex01a']

#     train_dataset, validation_dataset, test_dataset = createDataSet(marking)

#     val_loader = torch.utils.data.DataLoader(validation_dataset, batch_size=TrainGlobalConfig.batch_size,num_workers=0, #num_workers=0じゃないとkaggleではおちる
#             shuffle=False,sampler=SequentialSampler(validation_dataset),pin_memory=False,collate_fn=collate_fn,)

#     best_ckpt_path=glob(CNF['bestCKPT'])[0] if 'bestCKPT' in CNF.keys() else glob(f"effdet-*/*best*")[0]
    
#     # load ckpt ,make prediction
#     net = get_net(isTrn=False)
#     net.to(device)
#     fitter = Fitter(model=net, device=device, config=TrainGlobalConfig)
#     fitter.load(best_ckpt_path)
#     raw_pred=fitter.predict(val_loader,score_threshold=0.5)
#     iou_thresholds = [x for x in np.arange(0.5, 0.76, 0.05)]
#     scores=[p['scores'] for p in raw_pred]
#     # reshape pred/gt boxes 
#     amp = 1024/CNF['nnimSize']
#     pred_boxes=[ ((amp*p['boxes']).astype(np.int32).clip(min=0, max=1023))[p['scores'].argsort()[::-1]] for p in raw_pred ]
#     gt_boxes=_flattenList([[ _swapXY((amp*tgt['boxes']).cpu().numpy()).astype(np.int32) for tgt in val_batch[1] ] for val_batch in list(val_loader)])
#     scores=[p['scores'] for p in raw_pred]

#     from wheatutil import calculate_image_precision
#     metrics=np.stack([ np.array(calculate_image_precision(gt_box,pred_box, thresholds = iou_thresholds, form = 'pascal_voc')) for gt_box,pred_box in zip(gt_boxes,pred_boxes) ])
#     metrics={'cmp_scores':metrics[:,0],'precisions':metrics[:,1],'recalls':metrics[:,2] }

#     fig, ax = plt.subplots(1, 3, figsize=(15, 3))
#     for idx,metricName in enumerate([ 'cmp_scores','precisions','recalls' ]):

#         ax[idx].hist(metrics[metricName],bins=50)
#         ax[idx].axvline(np.mean(metrics[metricName]),c='r');
#         ax[idx].set_title(f"MEAN {metricName} {np.mean(metrics[metricName]):.5f}");

#     metrics.update({'image_id':[ _d[2] for _d in validation_dataset ]})
#     df=pd.DataFrame(metrics).merge(marking,how='left',on='image_id').drop(columns=['fold'])
#     _save(metrics,f"validation_score_{CNF['id']}.pkl")
#     print(df.groupby('source').mean()[['cmp_scores','precisions','recalls']])
    

    

# CV

In [ ]:
if CNF['checkPredcitions']:
    print(f"{np.mean(metrics['cmp_scores']):.4f}")

In [ ]:
if CNF['checkPredcitions']:
    
    NCOL=4
    fig, ax = plt.subplots(9, NCOL, figsize=(18, 12*NCOL))
    plt.subplots_adjust(left=0.005,right=0.995,bottom=0.005,top=0.995,wspace=0.005,hspace=0.005)
    for metric_idx,metricName in enumerate([ 'cmp_scores','precisions','recalls' ]):

        dataIds={}
        dataIds['low'] = metrics[metricName].argsort()[:NCOL]
        dataIds['mid'] = metrics[metricName].argsort()[len(metrics[metricName])//2:(NCOL+len(metrics[metricName])//2)]
        dataIds['high']= metrics[metricName].argsort()[-NCOL:][::-1]

        for quality_idx,(qualityName,ids) in enumerate(dataIds.items()):

            for col_idx,image_id in enumerate(ids):
                image=validation_dataset[image_id][0].permute(1,2,0).cpu().numpy()
                for box in (gt_boxes[image_id]/amp).astype(np.int32):
                    cv2.rectangle(image, (box[0], box[1]), (box[2], box[3]), (1, 1, 1), 6) 
                for box_idx,box in enumerate((pred_boxes[image_id]/amp).astype(np.int32)):
                    cv2.rectangle(image, (box[0], box[1]), (box[2], box[3]), (1, 0, 0), (scores[image_id][box_idx]>iou_thresholds).sum()) 

                ax[metric_idx*3+quality_idx][col_idx].imshow(image)
                ax[metric_idx*3+quality_idx][col_idx].set_axis_off()
                ax[metric_idx*3+quality_idx][col_idx].set_title(f"{metricName}-{qualityName}")

    

# Inference

In [ ]:
def make_predictions(images,net, score_threshold=0.22):
    images = torch.stack(images).cuda().float()
    predictions = []
    with torch.no_grad():
        det = net(images, torch.tensor([1]*images.shape[0]).float().cuda())
        for i in range(images.shape[0]):
            boxes = det[i].detach().cpu().numpy()[:,:4]    
            scores = det[i].detach().cpu().numpy()[:,4]
            indexes = np.where(scores > score_threshold)[0]
            boxes = boxes[indexes]
            boxes[:, 2] = boxes[:, 2] + boxes[:, 0]
            boxes[:, 3] = boxes[:, 3] + boxes[:, 1]
            predictions.append({
                'boxes': boxes[indexes],
                'scores': scores[indexes],
            })
    return [predictions]

def run_wbf(predictions, image_index, image_size=CNF['nnimSize'], iou_thr=0.44, skip_box_thr=0.43, weights=None):
    boxes = [(prediction[image_index]['boxes']/(image_size-1)).tolist()  for prediction in predictions]
    scores = [prediction[image_index]['scores'].tolist()  for prediction in predictions]
    labels = [np.ones(prediction[image_index]['scores'].shape[0]).tolist() for prediction in predictions]
    boxes, scores, labels = weighted_boxes_fusion(boxes, scores, labels, weights=None, iou_thr=iou_thr, skip_box_thr=skip_box_thr)
    boxes = boxes*(image_size-1)
    return boxes, scores, labels

def format_prediction_string(boxes, scores):
    pred_strings = []
    for j in zip(scores, boxes):
        pred_strings.append("{0:.4f} {1} {2} {3} {4}".format(j[0], j[1][0], j[1][1], j[1][2], j[1][3]))
    return " ".join(pred_strings)

In [ ]:
def infer():
    
    test_df=pd.DataFrame([path.split('/')[-1][:-4] for path in glob(f"{IMAGE_ROOT_PATH}/*jpg")],columns=['image_id'])
    test_dataset = DatasetRetriever(jigsaw=False,marking=test_df,transforms=get_test_transforms(),test=True,)
    tst_loader = DataLoader(test_dataset,batch_size=2,shuffle=False,num_workers=0,drop_last=False,collate_fn=collate_fn)
    net = get_net(isTrn=False).to(device)
    best_ckpt_path=glob(CNF['bestCKPT'])[0] if 'bestCKPT' in CNF.keys() else glob(f"effdet-*/*best*")[0]
    checkpoint = torch.load(best_ckpt_path, map_location=device) # GPUの時にチェックしてない
    net.model.load_state_dict(checkpoint['model_state_dict'])

    for j, (images, image_ids) in enumerate(tst_loader):
        break

    predictions = make_predictions(images,net)

    i = 0
    sample = images[i].permute(1,2,0).cpu().numpy()

    boxes, scores, labels = run_wbf(predictions, image_index=i)
    boxes = boxes.astype(np.int32).clip(min=0, max=511)

    fig, ax = plt.subplots(1, 1, figsize=(16, 8))

    for box in boxes:
        cv2.rectangle(sample, (box[0], box[1]), (box[2], box[3]), (1, 0, 0), 1)

    ax.set_axis_off()
    ax.imshow(sample);
    
    
    results = []
    ret_boxes = {};ret_scores = {}
    amp = 1024/CNF['nnimSize']
    
    for images, image_ids in tst_loader:
        predictions = make_predictions(images,net)
        for i, image in enumerate(images):
            boxes, scores, labels = run_wbf(predictions, image_index=i)
            boxes = (boxes*amp).astype(np.int32).clip(min=0, max=1023)
            image_id = image_ids[i]

            
            boxes[:, 2] = boxes[:, 2] - boxes[:, 0]
            boxes[:, 3] = boxes[:, 3] - boxes[:, 1]

            result = {
                'image_id': image_id,
                'PredictionString': format_prediction_string(boxes, scores),
            }
            results.append(result)
            ret_boxes[image_id]=boxes
            ret_scores[image_id]=scores

    test_df = pd.DataFrame(results, columns=['image_id', 'PredictionString'])
    
    return test_df,ret_boxes,ret_scores
    
    


In [ ]:
if CNF['infer']:
    
    # global name IMAGE_ROOT_PATH
    IMAGE_ROOT_PATH =  '../input/global-wheat-detection/test'
    
    test_df,_,_ = infer()#['image_id', 'PredictionString']
    test_df.to_csv('submission.csv', index=False)
    test_df.head()

In [ ]:
if CNF['createPseudo']:
    
    # global name IMAGE_ROOT_PATH
    #IMAGE_ROOT_PATH =  '../input/external01a/external01a'
    IMAGE_ROOT_PATH =  '../input/global-wheat-detection/train'
    
    pseudo_df, bboxes, scores = infer()
    
    _df=[]
    for image_id in bboxes.keys():

        _tmp_df = pd.DataFrame([ f"[{box[0]},{box[1]},{box[2]},{box[3]}]" for box in bboxes[image_id] ],columns=['bbox'])
        _tmp_df['image_id'] = image_id
        _tmp_df['score'] = scores[image_id]
        _df.append(_tmp_df)

    pseudo_df = pd.concat(_df)
    pseudo_df['width']=1024
    pseudo_df['height']=1024
    pseudo_df['source']='ex01b'
    #pseudo_df=pseudo_df[['image_id','width','height','bbox','source']]
    pseudo_df=pseudo_df[['image_id','width','height','bbox']]
    pseudo_df.to_csv(f"pred_bbox_{CNF['id']}.csv",index=False)
    
    


In [ ]:
if False:
    !mkdir data
    for _fold in range(2):
        !mkdir data/fold{_fold}
        for _src in SRC_IDS:
            !mkdir data/fold{_fold}/{_src}

            for image_id in df_folds[(df_folds.fold==_fold)&(df_folds.source==_src)].index:

                print(image_id)

                !cp {IMAGE_ROOT_PATH}/{image_id}.jpg data/fold{_fold}/{_src}/
            
        